In [1]:
from read_input import *
import codecs

# Frequency input
Put together the letter and bigram frequency files based on the data that comes from Mathieu. Distributes the frequency of combined characters that are not in the to-be-mapped character set of the scenario and filters out characters that are irrelevant.

In [2]:
#scenario doesn't matter, just need one 
scenario="scenario3"
character_set = "set1"
print(character_set)
letter_file = "input/resources/Code_all_Avg_freqs_1.tsv"
bigram_file = "input/resources/Code_all_Avg_freqs_2.tsv"
derive_probabilities_from_raw_values(letter_file, bigram_file, scenario, character_set,  "_code")

letter_file = "input/resources/Twitter_all_Avg_freqs_1.tsv"
bigram_file = "input/resources/Twitter_all_Avg_freqs_2.tsv"
derive_probabilities_from_raw_values(letter_file, bigram_file, scenario,character_set, "_twitter")

letter_file = "input/resources/Formals_all_Avg_freqs_1.tsv"
bigram_file = "input/resources/Formals_all_Avg_freqs_2.tsv"
derive_probabilities_from_raw_values(letter_file, bigram_file, scenario,character_set, "_formal")


set1
No frequency for /d
No frequency for ‘
No frequency for -d
No frequency for £
No frequency for §
No frequency for «
No frequency for ‰
No frequency for Ĳ
No frequency for ·
No frequency for »
No frequency for _d
No frequency for ¿
No frequency for Ç
No frequency for Ŋ
No frequency for ¸d
No frequency for Œ
No frequency for ×
No frequency for ß
No frequency for ≤
No frequency for ≠
No frequency for ˝d
No frequency for ç
No frequency for ÷
No frequency for ₩
No frequency for √
No frequency for ¤
No frequency for €
No frequency for ı
No frequency for °
No frequency for ¼
No frequency for À
No frequency for ≃
No frequency for °d
No frequency for †
No frequency for È
No frequency for Ð
No frequency for à
No frequency for è
No frequency for ˘d
No frequency for ˉd
No frequency for ð
No frequency for Ə
No frequency for ˙d
No frequency for θ2
No frequency for ‚
No frequency for „
No frequency for ¡
No frequency for ¥
No frequency for ©
No frequency for ±
No frequency for İ
No frequency for

({u'!': 0.00064068628724737065,
  u'#': 3.7845508335580451e-05,
  u'$': 7.542541986110475e-06,
  u'%': 0.00020903136954654518,
  u'&': 8.6606905688355331e-05,
  u'(': 0.0014715860769673153,
  u')': 0.0016172403607895491,
  u'*': 0.00042418368352249437,
  u'+': 6.2943002649076159e-05,
  u',': 0.0089846235060225665,
  u',d': 1.2402594005620963e-06,
  u'-': 0.0024769937304952201,
  u'-d': 1.1861276640314633e-09,
  u'.': 0.0077529147422787294,
  u'.d': 5.3937810017855544e-07,
  u'/': 0.00053620410352974758,
  u'/d': 2.7376011536116353e-09,
  u'0': 0.0028186952907733511,
  u'1': 0.0026837728874987386,
  u'2': 0.0021147019370857598,
  u'3': 0.00093386564165610399,
  u'4': 0.00077366200570420744,
  u'5': 0.00081403292568375135,
  u'6': 0.00066984185125278073,
  u'7': 0.00067332184210486365,
  u'8': 0.0007622144493819198,
  u'9': 0.0015010197771073654,
  u':': 0.0012762837367137816,
  u';': 0.00046545018059401957,
  u'<': 5.1885237778033486e-06,
  u'=': 4.8811601711224199e-05,
  u'>': 2.620801

### Single letter ranking based on probability, corpora weighted

In [11]:

corpus_weights = {"Formal": 0.5, "Twitter":0.3, "Code":0.2}

letter_files = ["input/resources/Formals_all_Avg_freqs_1.tsv","input/resources/Code_all_Avg_freqs_1.tsv", "input/resources/Twitter_all_Avg_freqs_1.tsv"]
p_single={}
p_bigrams = {}
for l_file in letter_files:
    print(l_file)
    print("weighting corpora")
    single = pd.read_csv(l_file, sep=" ", encoding="utf-8", index_col=0, quoting=3)
    single = single.to_dict()[u'frequency']
    #weight according to given weight:
    weight = 0
    for k,v in corpus_weights.iteritems():
        if k in l_file:
            weight = v
            break;
    if weight == 0:
        raise ValueError('no weight found for file: %s'%l_file)

    if len(p_single) == 0:
        p_single = {c:v*weight for c,v in single.iteritems()}     
    else:
        for c,v in single.iteritems():
            if c in p_single:
                p_single[c]+= v*weight
            else:
                p_single[c] = v*weight

input/resources/Formals_all_Avg_freqs_1.tsv
weighting corpora
input/resources/Code_all_Avg_freqs_1.tsv
weighting corpora
input/resources/Twitter_all_Avg_freqs_1.tsv
weighting corpora


In [34]:
df = pd.DataFrame.from_dict(p_single, orient="index")
df = df.reset_index()
df.columns=["letter", "frequency"]
df = df.sort_values(by="frequency", ascending=False)
with codecs.open("lists/letter_order_all.txt", 'w', encoding="utf-8") as f:
    for i in range(0,len(df)):
        f.write("%s\t%.8f\n"%(df.iloc[i,0], df.iloc[i,1]))
#df.to_csv("lists/letter_order_all.txt", sep="\t", encoding="utf-8", index=False)


# Distances
You only need to rerun this if you need to change the level cost

In [3]:
level_cost = {u'':0, u'Shift':1, u'Alt':2, u'Alt_Shift':3}

distance0, distance1 = read_distance_matrix(level_cost, recompute=0)
lines = ["%s %s %f\n"%(k1,k2,d) for (k1,k2), d in distance0.iteritems()]
with codecs.open("input\\distance0.txt", 'w', encoding="utf-8") as f:
    f.writelines(lines)
    
lines = ["%s %s %f\n"%(k1,k2,d) for (k1,k2), d in distance1.iteritems()]
with codecs.open("input\\distance1.txt", 'w', encoding="utf-8") as f:
    f.writelines(lines)

# Clean the Performance file 
to exclude transitions from B07 and B07_shift to non-letter keys and transitions from C10 to 